In [1]:
import pandas as pd
import json
import os
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

with open(os.path.join(os.getcwd(), '../config/cesco-poc-hc-service-key.json')) as f:
    hana_env_c = json.load(f)
    port_c = hana_env_c['port']
    user_c = hana_env_c['user']
    host_c = hana_env_c['host']
    pwd_c = hana_env_c['pwd']

cc = ConnectionContext(address=host_c, port=port_c, user=user_c, password=pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check hana_ml version
print(cc.hana_version())
#Check current schema
print(cc.get_current_schema())

2.2.2
2.21.24072600
4.00.000.00.1721126161 (fa/CE2024.14)
GEN_AI


In [3]:
#get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
        deployment_id = model,
        input = input
    )
    return response.data[0].embedding

In [39]:
#get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def run_vector_search_kr(query: str, metric="COSINE_SIMILARITY", k=5):

    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    elif metric == 'COSINE_SIMILARITY':
        sort = 'DESC'
        col = 'COS_SIM'

    query_vector = get_embedding(query)

    sql = '''SELECT TOP {k} "ProblemDescription","ProblemCategory", "ProblemKeyword",
    "Solution","SolutionDoc", "AdditionalInfo",
    "Code", "{metric}"("VectorProblem", TO_REAL_VECTOR('{qv}'))
    AS "{col}", "CreateDate", "UpdateDate"
    FROM "CESCO_PROBLEMSOLUTIONS"
    ORDER BY "{col}" {sort}
    '''.format(k=k, metric = metric, qv = query_vector, sort=sort, col=col)

    hdf = cc.sql(sql)
    df_context = hdf.collect()

    return df_context

In [40]:
query = "PDF 파일 열람 오류 해결방법 알려주세요"
df_context = run_vector_search_kr(query=query, metric="COSINE_SIMILARITY", k=5)
df_context

,ProblemDescription,ProblemCategory,ProblemKeyword,Solution,SolutionDoc,AdditionalInfo,Code,COS_SIM,CreateDate,UpdateDate
0,PDF 파일 열람 오류,MS Office 및 기타 프로그램,PDF,1.PDF 파일 형식에 맞는 연결 프로그램을 지정하세요. 연결 프로그램을 사용자가 ...,cesco3_0.pdf,Adobe Acrobat Reader DC 설치 방법 참고Adobe Acroba...,cesco3,0.910578,2024-08-30,2024-08-30
1,"PPT(파워포인트, PowerPoint) 파일 열람 오류",MS Office 및 기타 프로그램,PPT,1.윈도우→설정2.‘앱’ 클릭3.‘기본 앱’ 클릭4.‘파일 형식별 기본 앱 선택’ ...,cesco3_15.pdf,,cesco3,0.875997,2024-08-30,2024-08-30
2,"그룹웨어 메일, 게시판에 첨부된 문서 '열기'클릭 시 열람 오류",그룹웨어,그룹웨어,"1.크롬, 엣지 사이트 접속2.키보드 Ctrl + Shift + Delete 클릭3...",cesco5_11.pdf,,cesco5,0.853127,2024-08-30,2024-08-30
3,'DRM RunDLL / f_piimgr.exe 시스템 오류 해결 방법,보안프로그램,문서보안(DRM),1.윈도우 검색 (윈도우 키 + S) → DRM 응급복구 → 관리자 권한으로 실행2...,cesco8_7.pdf,,cesco8,0.845945,2024-08-30,2024-08-30
4,그룹웨어 게시물에 첨부된 문서 실행 시 빈 화면으로 보이는 문서열람 오류,MS Office 및 기타 프로그램,MS Office,1.문서 실행→파일→옵션2.보안 센터→보안 센터 설정3.제한된 보기→‘인터넷에서 가...,cesco3_6.pdf,,cesco3,0.844935,2024-08-30,2024-08-30


In [41]:
query = "PDF 파일 열람 오류 해결방법 알려주세요"
df_context = run_vector_search_kr(query=query, metric="L2DISTANCE", k=5)
df_context

,ProblemDescription,ProblemCategory,ProblemKeyword,Solution,SolutionDoc,AdditionalInfo,Code,L2D_SIM,CreateDate,UpdateDate
0,PDF 파일 열람 오류,MS Office 및 기타 프로그램,PDF,1.PDF 파일 형식에 맞는 연결 프로그램을 지정하세요. 연결 프로그램을 사용자가 ...,cesco3_0.pdf,Adobe Acrobat Reader DC 설치 방법 참고Adobe Acroba...,cesco3,0.422898,2024-08-30,2024-08-30
1,"PPT(파워포인트, PowerPoint) 파일 열람 오류",MS Office 및 기타 프로그램,PPT,1.윈도우→설정2.‘앱’ 클릭3.‘기본 앱’ 클릭4.‘파일 형식별 기본 앱 선택’ ...,cesco3_15.pdf,,cesco3,0.498002,2024-08-30,2024-08-30
2,"그룹웨어 메일, 게시판에 첨부된 문서 '열기'클릭 시 열람 오류",그룹웨어,그룹웨어,"1.크롬, 엣지 사이트 접속2.키보드 Ctrl + Shift + Delete 클릭3...",cesco5_11.pdf,,cesco5,0.541984,2024-08-30,2024-08-30
3,'DRM RunDLL / f_piimgr.exe 시스템 오류 해결 방법,보안프로그램,문서보안(DRM),1.윈도우 검색 (윈도우 키 + S) → DRM 응급복구 → 관리자 권한으로 실행2...,cesco8_7.pdf,,cesco8,0.555076,2024-08-30,2024-08-30
4,그룹웨어 게시물에 첨부된 문서 실행 시 빈 화면으로 보이는 문서열람 오류,MS Office 및 기타 프로그램,MS Office,1.문서 실행→파일→옵션2.보안 센터→보안 센터 설정3.제한된 보기→‘인터넷에서 가...,cesco3_6.pdf,,cesco3,0.556894,2024-08-30,2024-08-30


In [17]:
test_data = [
    "SC 법인 차량에 에어컨이 안나옵니다.",
    "차세대시스템 내 DB 구조가 궁금합니다.",
    "DATA를 DB에 올리는 방법이 궁금합니다.",
    "바퀴벌레 해충 동정 법이 궁금합니다.",
    "시궁쥐와 들쥐의 차이점이 궁금합니다.",
    "세스코 터치센터의 주소가 궁금해요.",
    "세스카페의 아메리카노 가격이 궁금합니다.",
    "세스코 터치센터 근처 버스정류장이 어디있나요",
    "터치센터에서 강일역 가능 방법이 궁금해요",
    "셔틀버스 탑승 시간을 알려주세요.",
    "출근 할 때 잠실역에서 셔틀버스 어디서 타는지 알고 싶어요.",
    "본사 직원 출근 시간이 궁금합니다.",
    "IT 서비스 데스크 직통 번호를 알려주세요.",
    "회사에서 지급한 노트북이 고장났어요.",
    "윈도우 설치 방법이 궁금합니다.",
    "세스톡 로그인이 안됩니다.",
    "연차 신청 방법이 궁금합니다.",
    "내 연차가 얼마나 남았는지 궁금해.",
    "이번 달 월급 명세서 확인하려면 어디로 가야하나요",
    "회사창립기념일이 언제인가요?"
]

In [44]:
s_list = []

for i in range(len(test_data)):
    df_context = run_vector_search_kr(query=test_data[i], metric="COSINE_SIMILARITY", k=5)
    
    v_s = pd.Series(test_data[i], name='query')
    for j in range(5):
        context_series = df_context.iloc[j]
        combined_series = pd.concat([v_s, context_series])
        s_list.append(combined_series)

s_df = pd.DataFrame(s_list)

s_df.to_csv('output4.csv', index=False, encoding='utf-8-sig')